<a href="https://colab.research.google.com/github/marlapinkert/TEWA2/blob/main/Individual_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Individual Assignment

The goal of the individual assignment is to perform simple preprocessing, quality checks, and first-level analysis for one singular subject. Check [here](https://docs.google.com/spreadsheets/d/1X4Xi0U9IghWKSXCLe1N6RU-Imar81wNs6ZWjaYIQ5Bg/edit?usp=sharing) to find out which subject is yours to analyse.

We will use the "NYU Slow Flanker" dataset from [OpenNeuro](https://openneuro.org/datasets/ds000102/versions/00001). In this study, 26 healthy adults underwent MRI scanning while performing an event-related Eriksen Flanker task.

## 1. Preprocessing
Before we can start with our analyses, we have to preprocess our data. As we did in the Practicals, we will use Neurodesk, which will provide us with the environment needed to be able to access software such as FSL.

### Set-up environment
The very first steps consist of downloading data and preparing our environment - just run the two cells below. You do not need to change any of the code in there! Remember: If your runtime disconnects, your files get lost and you have to rerun these cells.

In [ ]:
# download dataset (all subjects)
!mkdir /data
!wget 'https://ucloud.univie.ac.at/index.php/s/qKFm0f0Q98etPNG/download' -O /content/ds000102.zip
!unzip /content/ds000102.zip -d /data

In [ ]:
import os
os.environ["LD_PRELOAD"] = "";
os.environ["APPTAINER_BINDPATH"] = "/content"
os.environ["LMOD_CMD"] = "/usr/share/lmod/lmod/libexec/lmod"

!curl -J -O https://raw.githubusercontent.com/NeuroDesk/neurocommand/main/googlecolab_setup.sh
!chmod +x googlecolab_setup.sh
!./googlecolab_setup.sh
os.environ["MODULEPATH"] = ':'.join(map(str, list(map(lambda x: os.path.join(os.path.abspath('/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/'), x),os.listdir('/cvmfs/neurodesk.ardc.edu.au/neurodesk-modules/')))))

import lmod
await lmod.load('fsl/6.0.4')


os.environ["FSLOUTPUTTYPE"] = "NIFTI_GZ"

%cd /data

### Preprocessing the Functional File
Now that the environment is set-up, you can start with the preprocessing. We are keeping it "simple" - We will not "normalize" our file to the standard template. As we only do first-level analysis, this is not a problem. Your task is to:

1. **Calculate additional motion parameters** (`fsl.MotionOutliers`) -> This should be done on the raw file
2. **Perform slicetiming correction** (`fsl.SliceTimer`) -> This should be done on the raw file.
  
    **Note:** Although this information is lacking in the NIfTI header, we know from [Mennes et al. (2011)]( https://doi.org/10.1016/j.neuroimage.2010.10.046) that the study used an interleaved acquisition order. Make sure to specify this!

3. **Perform Motion correction** (`fsl.MCFLIRT`) -> This should be done on the slicetime-corrected file.

    **Note:** Make sure to save the motion parameters, so you can plot them later!

4. **Perform Smoothing** (`fsl.IsotropicSmooth`) -> This should be done on the motion-corrected file..

    **Note**: Make sure to specify an FWHM  of 6.

Tip 1: in the practicals, we used the `.outputs` attribute of Nipype interface results to access the paths to the processed files directly. You can also do this manually, but it is more tedious.

Tip 2: Using Nipype within Colab can cause issues if paths are not specified in a certain way. You are usually on the safe side if you provide the absolute path.



### Plotting Preprocessed Images
Plot both the raw and preprocessed images. Make sure to set a title for each. Comment on which differences you observe.

## 2. Quality Check - Motion Parameters

To recap the lecture, there are six motion parameters: 3 corresponding to translation (in x, y, z direction, respectively) and 3 corresponding to rotation (pitch, roll, yaw, respectively).

During preprocessing, we made sure to calculate these motion parameters by using `fsl.MCFLIRT`. We also calculated additional parameters with `fsl.MotionOutliers`. You will now:

1. **Plot the motion parameters**.
  
  Note: Your result should look something like the plots shown in the lecture and practical - you should have one plot for translation and one for rotation.
2. Calculate the **mean and maximum framewise displacement**.
3. **Describe** what the plots tell you - how much and in which directions did the participant move? How much framewise displacement was there? Do feel like this was too much movement?

**Bonus**:

  Create an additional plot, plotting the motion parameters as you did before. However, this time, also plot at which timepoints participants had to "press the button". i.e., respond to the task. This information is contained in the events file (ends in .tsv). Tip: The onset is given in seconds, but your plot just uses the timepoints (TRs).

  Is there a relation between movement and task?

## 3. First-Level Analysis

Using the preprocessed functional file, perform a first-level analysis. The steps are:

1. **Set-up Model** (using Nilearn's ```FirstLevelModel```)

2. **Fit the model** to the data
  - use the events file stored in the data directory
  - use the motion parameters as additional regressors
  
  
3. Plot the **design matrix**
4. Define the **contrasts**
  - congruent_correct > baseline
  - incongruent_correct > baseline
  - incongruent-congruent
  - congruent-incongruent
  - motion parameters > baseline
5. Calculate a **z-statistics map** for each contrast
6. Calculate a **statistical threshold** for each contrast
7. **Plot** the z-statistics maps (using the subject's anatomical image as background)
